# 1、torch.utils.data.Dataset：定义自定义数据集
## ⚠️ 核心概念：Dataset 是什么？
`torch.utils.data.Dataset` 是一个 `抽象类` (abstract class)，它代表了一个数据集。在 `PyTorch` 中，所有自定义的数据集都应该继承这个类。

它的核心思想是 将数据的`具体存储`、`访问方式`与`模型的训练`过程`解耦`。

✅ 数据集 `(Dataset)`：\
负责回答两个最基本的问题：“我的数据集里一共有`多少个样本`？” (`__len__`) 和 “请`给我第 i 个样本`” (`__getitem__`)。它只关心单个数据样本的获取和预处理。

✅ 数据加载器 (`DataLoader`)：\
负责从 `Dataset` 中取出数据，并把它们打包成一个个批次 (`batch`)，同时还可以进行`数据打乱` (`shuffle`) 和`多进程加载`等操作，高效地喂给模型进行训练。

简单来说，`Dataset` 定义了数据的`来源`和`单一样本的处理方式`，而 `DataLoader` 则在此基础上`构建了高效的数据流`。

---

## 👉 如何使用 `Dataset`：三大要素
要创建一个自定义的 `Dataset`，你只需要`继承` `torch.utils.data.Dataset` 并`重写` (`override`) 以下三个方法：

### 🧭 `_ _init_ _`(self, ...): 构造函数。

- 作用：执行数据集的初始化操作。这通常包括加载数据索引（比如`图片路径`和`对应的标签`）、定义数据变换 (`transform`) 等。

- 建议：在这个阶段，`不要` 加载所有的数据到内存中（除非你的数据集非常小）。通常只加载元信息（metadata），比如文件路径列表，这样可以节省大量内存。

### 🧭 `_ _len_ _`(self): 返回数据集的样本总数。

- 作用：`DataLoader` 需要知道数据集的总大小，以便确定迭代的次数、如何进行索引以及如何生成批次。

- 实现：通常是返回你在 `__init__` 中加载的`索引列表的长度`。

### 🧭 `_ _getitem_ _`(self, index): 根据索引 `index`获取并返回一个数据样本。
 
- 作用：这是 `Dataset` 的核心。`DataLoader` 会根据需要，传入一个索引 `index`，这个方法则需要根据这个索引定位到具体的数据文件，读取它，进行必要的预处理（如`图像缩放`、`裁剪`、`归一化`、转换成 `Tensor` 等），最后返回处理好的数据样本（通常是一个元组，例如 (`data_tensor`, `label_tensor`)）。

- 关键：真正的数据加载和转换（I/O 操作和计算）发生在这里，实现了`“按需加载”`，非常高效。

---

### 代码示例：自定义一个图像数据集
假设我们有如下的文件夹结构，用于一个简单的猫狗分类任务：

```
data/
├── cats/
│   ├── cat.0.jpg
│   ├── cat.1.jpg
│   └── ...
└── dogs/
    ├── dog.0.jpg
    ├── dog.1.jpg
    └── ...
```
### 现在，我们来创建一个自定义的 `Dataset` 来加载这些数据。

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image # 用于读取图片

class CatsAndDogsDataset(Dataset):
    """自定义猫狗分类数据集"""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): 包含 'cats' 和 'dogs' 文件夹的根目录。
            transform (callable, optional): 应用于样本的可选变换。
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []  # 用于存储 (图片路径, 标签) 的列表

        # 为猫和狗分配标签
        # cats -> 0, dogs -> 1
        for label, class_name in enumerate(['cats', 'dogs']):
            class_dir = os.path.join(self.root_dir, class_name)
            for file_name in os.listdir(class_dir):
                if file_name.endswith(('.jpg', '.png', '.jpeg')):
                    img_path = os.path.join(class_dir, file_name)
                    self.samples.append((img_path, label))

    def __len__(self):
        """
        返回数据集中样本的总数。
        """
        return len(self.samples)

    def __getitem__(self, index):
        """
        根据索引 index 获取一个样本。
        """
        # 1. 从 self.samples 中获取图片路径和标签
        img_path, label = self.samples[index]

        # 2. 读取图片
        # 使用 'L' 转换为灰度图，'RGB' 转换为彩色图
        image = Image.open(img_path).convert('RGB')

        # 3. 如果定义了变换，则对图片进行变换
        if self.transform:
            image = self.transform(image)
            
        # 4. 将标签也转换为 Tensor (可选，但推荐)
        label = torch.tensor(label, dtype=torch.long)

        # 5. 返回处理好的图片 Tensor 和标签 Tensor
        return image, label

---

## ⚠️ Dataset 与 DataLoader 的协同工作
`Dataset `本身只是一个`数据访问的接口`，它一次只能通过 [] 索引返回一个样本。要实现高效的训练，我们需要` DataLoader`。

- `DataLoader` 会从 `Dataset` 中自动拉取数据，并完成以下关键工作：

    - `批量处理` (Batching)：将多个样本打包成一个批次 (batch)。
    
    - `数据打乱` (Shuffling)：在每个` epoch `开始时，随机打乱数据顺序，以增强模型的泛化能力。
    
    - `并行加载` (Parallel Loading)：使用多个子进程 (num_workers) 同步加载数据，避免数据加载成为 GPU 计算的瓶颈。

---

### 使用示例：

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

# 1. 定义数据变换
# 这里我们定义了一个简单的变换：将图片缩放到 224x224，然后转换为 Tensor
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 标准化
])

# 2. 实例化我们自定义的 Dataset
dataset_path = 'data/'
custom_dataset = CatsAndDogsDataset(root_dir=dataset_path, transform=data_transform)
print(f"数据集大小: {len(custom_dataset)}")

# 3. 实例化 DataLoader
# - dataset: 我们创建的数据集实例
# - batch_size: 每个批次包含的样本数
# - shuffle: 是否在每个 epoch 开始时打乱数据
# - num_workers: 用于数据加载的子进程数量
data_loader = DataLoader(dataset=custom_dataset, batch_size=32, shuffle=True, num_workers=4)

# 4. 在训练循环中使用 DataLoader
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    # DataLoader 是一个可迭代对象
    for batch_images, batch_labels in data_loader:
        # 在这里，batch_images 的形状通常是 (batch_size, channels, height, width)
        # batch_labels 的形状通常是 (batch_size)
        
        # 接下来就可以将这些批次数据送入模型进行训练
        # model(batch_images) ...
        
        print(f"  - 批次图像形状: {batch_images.shape}")
        print(f"  - 批次标签形状: {batch_labels.shape}")
        break # 这里只演示一个批次

---

### ⚠️ 两种类型的 Dataset
PyTorch 提供了两种主要类型的 Dataset：

1、Map-style Datasets:

- 我们上面实现的就属于这种。
- 它实现了` __getitem__() `和` __len__() `方法。
- 它代表了从索引` (integer)`到数据样本的映射` (map)`。
这是最常用的一种。

2、Iterable-style Datasets:

- 它实现了` __iter__() `方法。
- 它代表了数据样本的一个`可迭代对象` (iterable)，类似于 Python 的`生成器`。
- 当你无法事先知道数据集的总长度，或者数据是从流中读取时（例如，`从数据库`或`远程服务器持续读取`），这种类型非常有用。

---

## ✅ 总结
`torch.utils.data.Dataset` 是 PyTorch 中构建数据输入管道的基石，它定义了如何获取单个数据样本。

- 通过继承它并实现` __init__`、`__len__ `和` __getitem__ `这三个核心方法，你可以为任何类型的数据创建自定义的加载逻辑。

`Dataset` 必须与` DataLoader` 配合使用，`DataLoader` 在` Dataset `的基础上提供了批处理、数据打乱和并行加载等高级功能，是构建高效、可读性强的数据管道的标准做法。